In [54]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import sys,os,os.path
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

from PIL import Image
from collections import namedtuple, OrderedDict
import math

## Device labels data into test set and train set

In [15]:
# lists the files inside 'annotations' in a random order (not really random, by their hash value instead)
# Moves the first 400/2000 labels (20% of the labels) to the testing dir: `test_labels`
!ls labels/* | sort -R | head -40 | xargs -I{} mv {} test_labels/

# Moves the rest of labels '1600' labels to the training dir: `train_labels`
!ls labels/* | xargs -I{} mv {} train_labels/

sort: write failed: 'standard output': Broken pipe
sort: write error


## Convert txt annotations file into csv

In [55]:
IMG_DIR = "images"

def txt_to_csv(path):
    classes_names = []
    txt_list = []

    print("path: ",path)
    for label_file in glob.glob(path + '/*.txt'):
        file = open(label_file, 'r') 
    
        count = 0
        while True: 
            count += 1
            line = file.readline() 
  
            if not line: 
                break
#             print("Line{}: {}".format(count, line.strip()))
            
            #process
            char = line.split(' ')
            img_name = char[0]
            label = char[1]
            x1 = float(char[2])
            y1 = float(char[3])
            x2 = float(char[4])
            y2 = float(char[5])
            
            # remove useless head path
            char = char[0].split('/')
            if len(char) > 1:
                img_name = char[1]
                
            image = Image.open(os.path.join(IMG_DIR, img_name))
            width, height = image.size
                
            print(img_name)
#             print(label)
#             print(x1, " ", y1, " ", x2, " ", y2)
            
            classes_names.append(label)
            value = (img_name,
                   int(width),
                   int(height),
                   label,
                   int(x1),
                   int(y1),
                   int(x2),
                   int(y2))
            txt_list.append(value)
            
        file.close() 
    
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(txt_list, columns=column_name) 
    classes_names = list(set(classes_names))
    classes_names.sort()
    return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
    image_path = os.path.join(os.getcwd(), label_path)
    xml_df, classes = txt_to_csv(label_path)
    xml_df.to_csv(f'{label_path}.csv', index=None)
    print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

path:  train_labels
0222_00578_b.jpg
19062019_15h_1932.jpg
0021_06361_b.jpg
0233_06177_b.jpg
0508_07355_b.jpg
0128_02555_b.jpg
0444_04878_b.jpg
a17_2.jpg
a17_2.jpg
a17_2.jpg
12_2.jpg
12_2.jpg
40865.jpg
0426_06591_b.jpg
13623.jpg
0125_00127_b.jpg
0138_00331_b.jpg
0135_00503_b.jpg
0511_07101_b.jpg
0310_07150_b.jpg
Oto_in_030719_8h_FHD_10688.jpg
Oto_in_030719_8h_FHD_10688.jpg
0450_05266_b.jpg
17653.jpg
0309_05262_b.jpg
0508_06887_b.jpg
0430_04666_b.jpg
0256_07076_b.jpg
0119_02023_b.jpg
0137_01947_b.jpg
missdetect_784.jpg
missdetect_784.jpg
9390.jpg
33361.jpg
40_406.jpg
40_406.jpg
0121_02555_b.jpg
0246_05507_b.jpg
img_145901.jpg
0517_05164_b.jpg
20677.jpg
6904.jpg
0214_06062_b.jpg
0420_04686_b.jpg
25227.jpg
8960.jpg
0242_01591_b.jpg
0210_04510_b.jpg
9227.jpg
25154.jpg
0138_04710_b.jpg
0123_06125_b.jpg
a23_2.jpg
missdetect_3570.jpg
24696.jpg
38406.jpg
12431.jpg
missdetect_4046.jpg
0458_06419_b.jpg
0259_04193_b.jpg
missdetect_34.jpg
0346_06965_b.jpg
0124_00806_b.jpg
6990.jpg
Oto_in_030719_8h

missdetect_992.jpg
0258_04132_b.jpg
19_444.jpg
19_444.jpg
img_145616.jpg
img_145616.jpg
img_145616.jpg
0422_05807_b.jpg
3880.jpg
Oto_in_030719_8h_FHD_1120.jpg
Oto_in_030719_8h_FHD_1120.jpg
0159_00418_b.jpg
0204_07385_b.jpg
0129_00633_b.jpg
14255.jpg
29448.jpg
10690.jpg
0257_06012_b.jpg
5586.jpg
0234_04694_b.jpg
25136.jpg
0236_04241_b.jpg
0312_07010_b.jpg
Oto_in_030719_8h_FHD_15496.jpg
0251_06509_b.jpg
0148_00001_b.jpg
0134_00217_b.jpg
0244_06332_b.jpg
0427_01058_b.jpg
0207_06374_b.jpg
0318_06155_b.jpg
21104.jpg
0313_04096_b.jpg
0141_04084_b.jpg
9006.jpg
0317_04106_b.jpg
11278.jpg
0222_04772_b.jpg
0412_06438_b.jpg
13860.jpg
0137_02601_b.jpg
0115_01937_b.jpg
0004_06801_b.jpg
0313_04662_b.jpg
41225.jpg
40585.jpg
Oto_in_030719_8h_FHD_15844.jpg
0222_05681_b.jpg
0325_06296_b.jpg
0039_01689_b.jpg
6809.jpg
missdetect_2238.jpg
0104_08083_b.jpg
img_145635.jpg
img_145635.jpg
img_145635.jpg
img_145635.jpg
img_145635.jpg
0132_08092_b.jpg
0439_05845_b.jpg
40928.jpg
31_2.jpg
0442_05448_b.jpg
0258_041

0202_03024_b.jpg
0450_06376_b.jpg
36666.jpg
0138_05250_b.jpg
0134_02562_b.jpg
0443_04382_b.jpg
0424_07398_b.jpg
0319_08008_b.jpg
0206_07127_b.jpg
Oto_in_030719_8h_FHD_10100.jpg
Oto_in_030719_8h_FHD_10100.jpg
0456_06380_b.jpg
Oto_in_030719_8h_FHD_16348.jpg
7093.jpg
Oto_in_030719_8h_FHD_15092.jpg
28430.jpg
0137_02503_b.jpg
0216_04264_b.jpg
25226.jpg
26334.jpg
1_2.jpg
1_2.jpg
1_2.jpg
img_145608.jpg
img_145608.jpg
img_145608.jpg
img_145608.jpg
0254_08017_b.jpg
0201_05320_b.jpg
0519_04391_b.jpg
9584.jpg
0427_06378_b.jpg
0134_00660_b.jpg
21172.jpg
Oto_in_030719_8h_FHD_16844.jpg
0150_04096_b.jpg
0304_06912_b.jpg
0338_05880_b.jpg
21238.jpg
25156.jpg
Oto_in_030719_8h_FHD_100.jpg
0253_05732_b.jpg
0140_02549_b.jpg
0142_00769_b.jpg
10072019_8h_276.jpg
10072019_8h_276.jpg
0519_05715_b.jpg
0339_06244_b.jpg
40432.jpg
13716.jpg
14002.jpg
0428_04599_b.jpg
32473.jpg
0305_08299_b.jpg
0503_06352_b.jpg
40_326.jpg
40_326.jpg
40_326.jpg
0305_06157_b.jpg
0201_05010_b.jpg
0302_04345_b.jpg
0314_06109_b.jpg
0251

0248_06621_b.jpg
0249_00984_b.jpg
0128_06831_b.jpg
0520_04394_b.jpg
0238_04368_b.jpg
0302_08192_b.jpg
0435_02056_b.jpg
6808.jpg
0204_05256_b.jpg
0116_02019_b.jpg
0250_04426_b.jpg
0305_01307_b.jpg
4170.jpg
0206_05783_b.jpg
25155.jpg
0316_04240_b.jpg
0458_06268_b.jpg
34873.jpg
0154_02580_b.jpg
37278.jpg
30752.jpg
21150.jpg
17754.jpg
0131_00942_b.jpg
0124_00942_b.jpg
0421_08195_b.jpg
missdetect_4114.jpg
300.jpg
0108_00973_b.jpg
3590.jpg
15693.jpg
12277.jpg
0235_04862_b.jpg
9096.jpg
12244.jpg
17175.jpg
0138_00196_b.jpg
14020.jpg
0148_02583_b.jpg
29156.jpg
0512_04241_b.jpg
0505_04350_b.jpg
0219_05824_b.jpg
0215_05885_b.jpg
0428_07359_b.jpg
5722.jpg
Oto_in_030719_8h_FHD_1684.jpg
1_180.jpg
1_180.jpg
1_180.jpg
1_180.jpg
a13_232.jpg
a13_232.jpg
a13_232.jpg
41071.jpg
0312_05504_b.jpg
0512_08200_b.jpg
0519_06917_b.jpg
0433_07371_b.jpg
33221.jpg
out_19062019_15h30_6540.jpg
out_19062019_15h30_6540.jpg
out_19062019_15h30_6540.jpg
0256_08083_b.jpg
20707.jpg
0438_04055_b.jpg
40625.jpg
0252_08125_b.jpg

17755.jpg
0316_06813_b.jpg
Oto_in_030719_8h_FHD_104.jpg
7_1560.jpg
7_1560.jpg
0342_06180_b.jpg
0042_01746_b.jpg
11118.jpg
744.jpg
37550.jpg
0509_04070_b.jpg
29584.jpg
0405_05175_b.jpg
0319_06474_b.jpg
5515.jpg
0118_00754_b.jpg
4054.jpg
0516_05104_b.jpg
0433_04098_b.jpg
a21_2.jpg
0028_07031_b.jpg
0425_04576_b.jpg
0238_01814_b.jpg
0403_04247_b.jpg
5347.jpg
0125_02237_b.jpg
0143_05783_b.jpg
0257_04030_b.jpg
0258_05146_b.jpg
0251_04333_b.jpg
0505_05633_b.jpg
0505_04873_b.jpg
0356_04996_b.jpg
Oto_in_030719_8h_FHD_13744.jpg
Oto_in_030719_8h_FHD_13744.jpg
Oto_in_030719_8h_FHD_13744.jpg
0110_01967_b.jpg
0359_01806_b.jpg
0212_00289_b.jpg
0519_02272_b.jpg
Oto_in_030719_8h_FHD_12740.jpg
0450_06152_b.jpg
0114_00442_b.jpg
0100_00168_b.jpg
1_594.jpg
1_594.jpg
1_594.jpg
1_594.jpg
Oto_in_030719_8h_FHD_12112.jpg
Oto_in_030719_8h_FHD_12112.jpg
0233_05608_b.jpg
24827.jpg
0144_04987_b.jpg
32900.jpg
40591.jpg
0447_06967_b.jpg
0158_01702_b.jpg
0058_01153_b.jpg
0503_04455_b.jpg
0115_01928_b.jpg
11477.jpg
384

## Convert xml file into csv

In [51]:
#adjusted from: https://github.com/datitran/raccoon_dataset
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


## Generate TFRecord

In [56]:
#adjusted from: https://github.com/datitran/raccoon_dataset
from object_detection.utils import dataset_util
import tensorflow as tf
import cv2 

#change this to the base directory where your data/ is 
# data_base_url = '/gdrive/My Drive/raccoon/data/'

#location of images
image_dir = 'images/'

def class_text_to_int(row_label):
  if row_label == 'plate':
    return 1
  else:
    None

def split(df, group):
  data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
	  encoded_jpg = fid.read()
	encoded_jpg_io = io.BytesIO(encoded_jpg)
	image = Image.open(encoded_jpg_io)
	width, height = image.size
	filename = group.filename.encode('utf8')
	image_format = b'jpg'
	xmins = []
	xmaxs = []
	ymins = []
	ymaxs = []
	classes_text = []
	classes = []

	for index, row in group.object.iterrows():
		xmins.append(row['xmin'] / width)
		xmaxs.append(row['xmax'] / width)
		ymins.append(row['ymin'] / height)
		ymaxs.append(row['ymax'] / height)
		classes_text.append(row['class'].encode('utf8'))
		classes.append(class_text_to_int(row['class']))

	tf_example = tf.train.Example(features=tf.train.Features(feature={
		'image/height': dataset_util.int64_feature(height),
		'image/width': dataset_util.int64_feature(width),
		'image/filename': dataset_util.bytes_feature(filename),
		'image/source_id': dataset_util.bytes_feature(filename),
		'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		'image/format': dataset_util.bytes_feature(image_format),
		'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
	return tf_example

In [57]:
# %cd /gdrive/'My Drive'/Data/raccoon/data
#creates tfrecord for both csv's

for csv in ['train_labels', 'test_labels']:
#   print(data_base_url)
  writer = tf.io.TFRecordWriter(csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), csv + '.record')
  print('Successfully created the TFRecords: {}'.format(csv + '.record'))

Successfully created the TFRecords: train_labels.record
Successfully created the TFRecords: test_labels.record
